### Statement of the Problem/Requirements
1) Write an application to summarize the given text
2) The summary should be free of industrial jargons
3) Need to use any LLM from OpenAI
4) Do not expose the OpenAI API Key
5) Prepare a well documented jupyter notebook as well as script file

#### Import packages

In [5]:
import os
import re
import sys
import argparse

import openai
import tiktoken
from dotenv import load_dotenv

#### OpenAI Settings

In [3]:
PERSONA_PROMPT = "You are to act as an expert project manager."

SECTION_PROMPT = f"{PERSONA_PROMPT} Paraphrase each thought into bullet point statements. Do not include an intro or conclusion."
TOPIC_PROMPT = f"{PERSONA_PROMPT} Separate the following notes into sections by topic. Do not change the wording or order of notes."
SUMMARY_PROMPT = f"{PERSONA_PROMPT} Summarize the following meeting notes in Key Takeaways and Action Items. Key Takeaways and Action Items should not repeat each other."

TEMPERATURE = 0
OVERLAP = 50
SECTION_RESPONSE_MAX_TOKENS = 1024

#### Set up OpenAI API credentials and model name

In [6]:
load_dotenv()
API_KEY = os.getenv("OPENAI_API_KEY", None)
if API_KEY is None:
    print("Error: OPENAI_ORG_ID and OPENAI_API_KEY environment variables must be set.")
    sys.exit(1)
openai.api_key = API_KEY

#### Select Model

In [8]:
MODEL = "gpt-3.5-turbo"
SYSTEM_PROMPT = "You are a helpful assistant."

#### Get encoding

In [10]:
enc = tiktoken.get_encoding("gpt2")

#### Get input file

In [11]:
def get_input_text(input_file):
    try:
        with open(input_file, 'r') as f:
            text = f.read()
    except FileNotFoundError:
        print(f"Error: file '{input_file}' not found.")
        sys.exit(1)
    return text

#### Setting for processing the input file

In [13]:
def parse_arguments():
    parser = argparse.ArgumentParser(
        description="Process text file and create summaries using OpenAI.")
    parser.add_argument("input_file", help="The input text file to process.")
    parser.add_argument("-o", "--output_file", nargs="?",
                        help="The output file where the results will be saved. If omitted, the output file will be named the same as the input file, but appended with '_output' and always have a '.txt' extension.")
    parser.add_argument("-j", "--jargon_file", nargs="?", const="jargon.txt",
                        help="Replace jargon terms before processing text. Will check for jargon.txt in the current working directory unless another file location is specified.")
    parser.add_argument("-t", "--topics", nargs="?", const="prompt",
                        help="Sort notes by topic. Provide a comma-separated list of topics or use 'auto' to automatically generate topics. Default is 'prompt' which will ask for the list at runtime.")
    parser.add_argument("-s", "--summary", action="store_true",
                        help="Generate a summary of the notes and include in the output file.")
    return parser.parse_args()